In [16]:
import os

from __future__ import print_function

import json
import sys
import re

# This is not required if you've installed pycparser into
# your site-packages/ with setup.py
#
sys.path.extend(['.', '..'])

from pycparser import parse_file, c_ast
from pycparser.plyparser import Coord


RE_CHILD_ARRAY = re.compile(r'(.*)\[(.*)\]')
RE_INTERNAL_ATTR = re.compile('__.*__')


class CJsonError(Exception):
    pass


def memodict(fn):
    """ Fast memoization decorator for a function taking a single argument """
    class memodict(dict):
        def __missing__(self, key):
            ret = self[key] = fn(key)
            return ret
    return memodict().__getitem__


@memodict
def child_attrs_of(klass):
    """
    Given a Node class, get a set of child attrs.
    Memoized to avoid highly repetitive string manipulation
    """
    non_child_attrs = set(klass.attr_names)
    all_attrs = set([i for i in klass.__slots__ if not RE_INTERNAL_ATTR.match(i)])
    return all_attrs - non_child_attrs


def to_dict(node):
    """ Recursively convert an ast into dict representation. """
    klass = node.__class__

    result = {}

    # Metadata
    result['_nodetype'] = klass.__name__

    # Local node attributes
    for attr in klass.attr_names:
        result[attr] = getattr(node, attr)

    # Coord object
    if node.coord:
        result['coord'] = str(node.coord)
    else:
        result['coord'] = None

    # Child attributes
    for child_name, child in node.children():
        # Child strings are either simple (e.g. 'value') or arrays (e.g. 'block_items[1]')
        match = RE_CHILD_ARRAY.match(child_name)
        if match:
            array_name, array_index = match.groups()
            array_index = int(array_index)
            # arrays come in order, so we verify and append.
            result[array_name] = result.get(array_name, [])
            if array_index != len(result[array_name]):
                raise CJsonError('Internal ast error. Array {} out of order. '
                    'Expected index {}, got {}'.format(
                    array_name, len(result[array_name]), array_index))
            result[array_name].append(to_dict(child))
        else:
            result[child_name] = to_dict(child)

    # Any child attributes that were missing need "None" values in the json.
    for child_attr in child_attrs_of(klass):
        if child_attr not in result:
            result[child_attr] = None

    return result


def to_json(node, **kwargs):
    """ Convert ast node to json string """
    return json.dumps(to_dict(node), **kwargs)


def file_to_dict(filename):
    """ Load C file into dict representation of ast """
    ast = parse_file(filename, use_cpp=True,
            cpp_path='clang',
            cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return to_dict(ast)


def file_to_json(filename, **kwargs):
    """ Load C file into json string representation of ast """
    ast = parse_file(filename, use_cpp=True,
            cpp_path='clang',
            cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return to_json(ast, **kwargs)


def _parse_coord(coord_str):
    """ Parse coord string (file:line[:column]) into Coord object. """
    if coord_str is None:
        return None

    vals = coord_str.split(':')
    vals.extend([None] * 3)
    filename, line, column = vals[:3]
    return Coord(filename, line, column)


def _convert_to_obj(value):
    """
    Convert an object in the dict representation into an object.
    Note: Mutually recursive with from_dict.
    """
    value_type = type(value)
    if value_type == dict:
        return from_dict(value)
    elif value_type == list:
        return [_convert_to_obj(item) for item in value]
    else:
        # String
        return value


def from_dict(node_dict):
    """ Recursively build an ast from dict representation """
    class_name = node_dict.pop('_nodetype')

    klass = getattr(c_ast, class_name)

    # Create a new dict containing the key-value pairs which we can pass
    # to node constructors.
    objs = {}
    for key, value in node_dict.items():
        if key == 'coord':
            objs[key] = _parse_coord(value)
        else:
            objs[key] = _convert_to_obj(value)

    # Use keyword parameters, which works thanks to beautifully consistent
    # ast Node initializers.
    return klass(**objs)


def from_json(ast_json):
    """ Build an ast from json string representation """
    return from_dict(json.loads(ast_json))

In [17]:
folderPath = './Testdata/'
fileList = os.listdir(folderPath)

In [18]:
outfile = open('result.txt', "w")
for fileName in fileList:
    ast_dict = file_to_dict(folderPath + fileName)
    ast = from_dict(ast_dict)
    jsonString = to_json(ast,indent=4)
    jsonObject = json.loads(jsonString)
    outfile.write(fileName + "\n")
    json.dump(jsonObject, outfile, indent=4)
    outfile.write("\n\n")
    # print(fileName)
    # print(json.dumps(jsonObject, indent = 4))
    # print("----------------------------------------------------------------------------------------")

In [19]:
"""
AST Start
Go into ext
Go into decl
Go into body
Go into block items
for every Decl node in block itmes
    Get variable type
    If the variable is initialized
        Go into initialization
        If initalized to a constant
            get the constant
        Else if initialized to a variable and the variable is not a function argumaent
            get the variable from the dict
        Else if initialzied to an expression
            if all operands are constants
                evaluate the expression
            else if some of the operands are variables
                if some variables are unknown
                    set the value to 'unknown'
                else
                    get the variable values
                    evaluate the expression
            else
                set the value to 'unknown'
for every Assignment node in block items
    If Assignment['lvalue']['name'] in dict and dict[Assignment['lvalue']['name']]['type'] == 'int'
    Go into assignment
        If assigned to a constant
            get the constant
        Else if assigned to a variable and the variable is not a function argumaent
            get the variable from the dict
        Else if assigned to an expression
            if all operands are constants
                evaluate the expression
            else if some of the operands are variables
                if some variables are unknown
                    set the value to 'unknown'
                else
                    get the variable values
                    evaluate the expression
            else
                set the value to 'unknown'
"""

def createVarDict(blockItems):
    varDict = dict()
    for blockDict in blockItems:
        if blockDict['_nodetype'] == 'Decl':
            varDict[blockDict['name']] = {'type': blockDict['type']['type']['names'][0], 'value': 'null'}
            if 'init' in blockDict and bool(blockDict['init']):
                init = blockDict['init']
                if init['_nodetype'] == 'Constant':
                    varDict[blockDict['name']]['value'] = init['value']
                elif init['_nodetype'] == 'ID':
                    for var in varDict.keys():
                        if var == init['name'] and varDict[var]['value'] != 'null':
                            print(varDict[var]['value'])
                            varDict[blockDict['name']]['value'] = exec(varDict[blockDict['name']]['type'] + "(" + varDict[var]['value'] + ")") #or eval()
                elif init['_nodetype'] == 'BinaryOp':
                    if init['left']['_nodetype'] == 'Constant' and init['right']['_nodetype'] == 'Constant':
                        varDict[blockDict['name']]['value'] = exec(varDict[blockDict['name']]['type'] + "(" + init['left']['value'] + init['op'] + init['right']['value'] + ")")
                    elif init['left']['_nodetype'] == 'ID' or init['right']['_nodetype'] == 'ID':
                        leftVal = 'null'
                        rightVal = 'null'

                        if init['left']['_nodetype'] == 'ID':
                            for var in varDict.keys():
                                if var == init['left']['name']:
                                    leftVal = varDict[var]['value']
                        elif init['left']['_nodetype'] == 'Constant':
                            leftVal = init['left']['value']
                        
                        if init['right']['_nodetype'] == 'ID':
                            for var in varDict.keys():
                                if var == init['right']['name']:
                                    leftVal = varDict[var]['value']
                        elif init['right']['_nodetype'] == 'Constant':
                            leftVal = init['right']['value']
                        
                        if leftVal == 'null' or rightVal == 'null':
                            varDict[blockDict['name']]['value'] = 'null'
                        else:
                            varDict[blockDict['name']]['value'] = exec(varDict[blockDict['name']]['type'] + "(" + str(rightVal) + init['op'] + str(leftVal) + ")")

        elif blockDict['_nodetype'] == 'Assignment':
            varName = blockDict['lvalue']['name']
            assignDict = blockDict['rvalue']

            if assignDict['_nodetype'] == 'Constant':
                    varDict[varName]['value'] = assignDict['value']
            elif assignDict['_nodetype'] == 'ID':
                for var in varDict.keys():
                    if var == assignDict['name'] and varDict[var]['value'] != 'null':
                        print(varDict[var]['value'])
                        varDict[varName]['value'] = exec(varDict[varName]['type'] + "(" + varDict[var]['value'] + ")") #or eval()
            elif assignDict['_nodetype'] == 'BinaryOp':
                if assignDict['left']['_nodetype'] == 'Constant' and assignDict['right']['_nodetype'] == 'Constant':
                    varDict[varName]['value'] = exec(varDict[varName]['type'] + "(" + assignDict['left']['value'] + assignDict['op'] + assignDict['right']['value'] + ")")
                elif assignDict['left']['_nodetype'] == 'ID' or assignDict['right']['_nodetype'] == 'ID':
                    leftVal = 'null'
                    rightVal = 'null'

                    if assignDict['left']['_nodetype'] == 'ID':
                        for var in varDict.keys():
                            if var == assignDict['left']['name']:
                                leftVal = varDict[var]['value']
                    elif assignDict['left']['_nodetype'] == 'Constant':
                        leftVal = assignDict['left']['value']
                    
                    if assignDict['right']['_nodetype'] == 'ID':
                        for var in varDict.keys():
                            if var == assignDict['right']['name']:
                                leftVal = varDict[var]['value']
                    elif assignDict['right']['_nodetype'] == 'Constant':
                        leftVal = assignDict['right']['value']
                    
                    if leftVal == 'null' or rightVal == 'null':
                        varDict[varName]['value'] = 'null'
                    else:
                        varDict[varName]['value'] = exec(varDict[varName]['type'] + "(" + str(rightVal) + assignDict['op'] + str(leftVal) + ")")
        # elif blockDict['_nodetype'] == 'If' or blockDict['_nodetype'] == 'While':
        #     varDict.update(createVarDict(blockDict))
    return varDict

In [20]:
def detectRisks(json_object, paramList, varDict, funcDict):
    if type(json_object) == dict:
        for key in json_object:
            if key == '_nodetype' and json_object[key] != 'FileAST':
                #print(json_object[key])
                if json_object[key] == 'FuncDef':
                    #Create paramList
                    paramList = list()
                    if json_object['decl']['type']['args'] != None:
                        for param in json_object['decl']['type']['args']['params']:
                            paramList.append(param['name'])

                    #Create varDict
                    varDict = createVarDict(json_object['body']['block_items'])
                    #Add function return type to funcDict
                    funcDict.update({json_object['decl']['name']: json_object['decl']['type']['type']['type']['names'][0]})
                elif json_object[key] == 'Decl' and 'init' in json_object and bool(json_object['init']) and 'op' in json_object['init']:
                    operation = json_object['init']['op']
                    if operation == '+' or operation == '-' or operation == '*':
                        if (json_object['init']['left']['name'] in paramList or json_object['init']['right']['name'] in paramList) and json_object['type']['type']['names'][0] == 'int':
                            match = re.search('(:[0-9]+:)', json_object['init']['coord'])
                            lineNum = match.string.strip(':')
                            print('Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters used in assignment)')
                            print('Revise line ' + str(lineNum))
                            print()
                elif json_object[key] == 'Return' and list(funcDict.values())[-1] == 'int':
                    #Case for 'return b - c' where return is an int, where b or c are function arguments
                    if 'op' in json_object['expr'] and (json_object['expr']['op'] == '+' or json_object['expr']['op'] == '-' or json_object['expr']['op'] == '*') and (json_object['expr']['left']['name'] in paramList or json_object['expr']['right']['name'] in paramList):
                        match = re.search('(:[0-9]+:)', json_object['coord'])
                        lineNum = match.string.strip(':')
                        print('Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters returned)')
                        print('Revise line ' + str(lineNum))
                        print()
                    #Case for 'return a' where return is an int and 'a' is a function argument
                    elif 'name' in json_object['expr'] and json_object['expr']['name'] in paramList:
                        match = re.search('(:[0-9]+:)', json_object['coord'])
                        lineNum = match.string.strip(':')
                        print('Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters returned)')
                        print('Revise line ' + str(lineNum))
                        print()
            detectRisks(json_object[key], paramList, varDict, funcDict)
    # if type(json_object) == dict:
    #     for key in json_object:
    #         if key == 'ext':
    #             for item in json_object[key]:
    #                 if item['_nodetype'] == 'FuncDef':
                        
    #                     #Create paramList
    #                     paramList = list()
    #                     if item['decl']['type']['args'] != None:
    #                         for param in item['decl']['type']['args']['params']:
    #                             paramList.append(param['name'])

    #                     #Create varDict
    #                     varDict = createVarDict(item['body']['block_items'])
                           

    #                     finalBlock = item['body']['block_items'][-1]
    #                     if item['decl']['type']['type']['type']['names'][0] == 'int' and finalBlock['_nodetype'] == 'Return':
    #                         #Case for 'return b - c' where return is an int, where b or c are function arguments
    #                         if 'op' in finalBlock['expr'] and (finalBlock['expr']['op'] == '+' or finalBlock['expr']['op'] == '-' or finalBlock['expr']['op'] == '*') and (finalBlock['expr']['left']['name'] in paramList or finalBlock['expr']['right']['name'] in paramList):
    #                             match = re.search('(:[0-9]+:)', finalBlock['coord'])
    #                             lineNum = match.string.strip(':')
    #                             print('Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters returned)')
    #                             print('Revise line ' + str(lineNum))
    #                             print()
    #                         #Case for 'return a' where return is an int and 'a' is a function argument
    #                         elif 'name' in finalBlock['expr'] and finalBlock['expr']['name'] in paramList:
    #                             match = re.search('(:[0-9]+:)', finalBlock['coord'])
    #                             lineNum = match.string.strip(':')
    #                             print('Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters returned)')
    #                             print('Revise line ' + str(lineNum))
    #                             print()
                        
    #                     for blockDict in item['body']['block_items']:
    #                         #Case for 'int a = b - c', where b or c are function arguments
    #                         if blockDict['_nodetype'] == 'Decl' and 'init' in blockDict and bool(blockDict['init']) and 'op' in blockDict['init']:
    #                             operation = blockDict['init']['op']
    #                             if operation == '+' or operation == '-' or operation == '*':
    #                                 if (blockDict['init']['left']['name'] in paramList or blockDict['init']['right']['name'] in paramList) and blockDict['type']['type']['names'][0] == 'int':
    #                                     match = re.search('(:[0-9]+:)', blockDict['init']['coord'])
    #                                     lineNum = match.string.strip(':')
    #                                     print('Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters used in assignment)')
    #                                     print('Revise line ' + str(lineNum))
    #                                     print()
    #                         #Case for 'a = b - c' where a is an int and b or c are function arguments
    #                         elif blockDict['_nodetype'] == 'Assignment' and 'rvalue' in blockDict and 'op' in blockDict['rvalue'] and (blockDict['rvalue']['op'] == '+' or blockDict['rvalue']['op'] == '-' or blockDict['rvalue']['op'] == '*') and 'name' in blockDict['rvalue']['left'] and 'name' in blockDict['rvalue']['right'] and (blockDict['rvalue']['left']['name'] in paramList or blockDict['rvalue']['right']['name'] in paramList):
    #                             flaggedVar = blockDict['lvalue']['name']
    #                             for declDict in item['body']['block_items']:
    #                                 if declDict['_nodetype'] == 'Decl' and declDict['name'] == flaggedVar and declDict['type']['type']['names'][0] == 'int':
    #                                     match = re.search('(:[0-9]+:)', blockDict['coord'])
    #                                     lineNum = match.string.strip(':')
    #                                     print('Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters used in initialization)')
    #                                     print('Revise line ' + str(lineNum))
    #                                     print()
            #else:
                #Case for 'b + c < 0' where b, c are positive integers
                # if blockDict['_nodetype'] == 'Assignment' and 'op' in blockDict['rvalue'] and blockDict['rvalue']['op'] == '+':
                #     operandList = {'op1': {'name': '', 'value': 0}, 'op2': {'name': '', 'value': 0}}
                #     rDict = blockDict['rvalue']
                #     if rDict['left']['_nodetype'] == 'ID':
                #         operandList['op1']['name'] = rDict['left']['name']
                #     if rDict['right']['_nodetype'] == 'ID':
                #         operandList['op2']['name'] = rDict['right']['name']
                #     for declDict in item['body']['block_items']:
                #          if declDict['_nodetype'] == 'Decl' and (operandList['op1']['name'] == declDict['name'] or operandList['op2']['name'] == declDict['name']) and declDict['type']['type']['names'][0] == 'int':
                #              if operandList['op1']['name'] == declDict['name']:
    elif type(json_object) == list:
        for item in json_object:
            detectRisks(item, paramList, varDict, funcDict)

In [21]:
for fileName in fileList:
    ast_dict = file_to_dict(folderPath + fileName)
    ast = from_dict(ast_dict)
    jsonString = to_json(ast,indent=4)
    jsonObject = json.loads(jsonString)
    print(fileName)
    detectRisks(jsonObject, list(), dict(), dict())
    print("----------------------------------------------------------------------------------------")

test1.c
Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters returned)
Revise line ./Testdata/test1.c:3:5

----------------------------------------------------------------------------------------
test2.c
Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters used in assignment)
Revise line ./Testdata/test2.c:3:13

Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters used in assignment)
Revise line ./Testdata/test2.c:4:13

Possible vunerability "CWE-190 Integer Overflow or Wraparound" detected. (Case: Function parameters returned)
Revise line ./Testdata/test2.c:9:5

----------------------------------------------------------------------------------------
test3.c
----------------------------------------------------------------------------------------
test4.c
----------------------------------------------------------------------------------------
test5.

In [22]:
"""
AST Start
Laterally find ext
Laterally find body
Go into body
Laterally find block_items
Go into block_items
    Find index with "init": not NULL
        If ("name": is a var int) AND (init['op'] is +,-,*) AND ((init['left']['_nodetype'] is 'ID') OR (init['right']['_nodetype'] is 'ID'))
            Flag the line
    Find index with "_nodetype": Assignment
        If (lvalue['name'] is a var int) AND (rvalue['op'] is +,-,*) AND ((left['_nodetype'] is 'ID') OR (right['_nodetype'] is 'ID'))
            Flag the line

"""

"""
AST Start
Laterally find ext
Laterally find body
Go into body
Laterally find block_items
Go into block_items
Go through indicies
    If '_nodetype' is 'Assignment' AND 'rvalue'['op'] is +,-,* AND 'rvalue'['left']['_nodetype'] is 'ID' OR 'rvalue'['right']['_nodetype'] is 'ID'
        Store block_items[index]['lvalue']['name'] into var
        goto block_items[0] and go through indicies
            If '_nodetype' is 'Decl' AND 'name' is var AND 'type'['type']['names'][0] is int
                Flag the line
"""

"""
AST Start
Go into ext
Go into decl
Go into body
Go into block items
for every Decl node in block itmes
    Get variable type
    If the variable is initialized
        Go into initialization
        If initalized to a constant
            get the constant
        Else if initialized to a variable and the variable is not a function argumaent
            get the variable from the dict
        Else if initialzied to an expression
            if all operands are constants
                evaluate the expression
            else if some of the operands are variables
                if some variables are unknown
                    set the value to 'unknown'
                else
                    get the variable values
                    evaluate the expression
            else
                set the value to 'unknown'
for every Assignment node in block items
    If Assignment['lvalue']['name'] in dict and dict[Assignment['lvalue']['name']]['type'] == 'int'
    Go into assignment
        If assigned to a constant
            get the constant
        Else if assigned to a variable and the variable is not a function argumaent
            get the variable from the dict
        Else if assigned to an expression
            if all operands are constants
                evaluate the expression
            else if some of the operands are variables
                if some variables are unknown
                    set the value to 'unknown'
                else
                    get the variable values
                    evaluate the expression
            else
                set the value to 'unknown'
"""


"\nAST Start\nGo into ext\nGo into decl\nGo into body\nGo into block items\nfor every Decl node in block itmes\n    Get variable type\n    If the variable is initialized\n        Go into initialization\n        If initalized to a constant\n            get the constant\n        Else if initialized to a variable and the variable is not a function argumaent\n            get the variable from the dict\n        Else if initialzied to an expression\n            if all operands are constants\n                evaluate the expression\n            else if some of the operands are variables\n                if some variables are unknown\n                    set the value to 'unknown'\n                else\n                    get the variable values\n                    evaluate the expression\n            else\n                set the value to 'unknown'\nfor every Assignment node in block items\n    If Assignment['lvalue']['name'] in dict and dict[Assignment['lvalue']['name']]['type'] == 'int'\n

In [23]:
def bool1():
    print("bool1")
    return False
def bool2():
    print("bool2")
    return True

if bool1() and bool2():
    1 + 1

bool1


In [24]:
d = {}
d['a'] = 1
d['b'] = 2
d['c'] = 3
d['d'] = 4

arr = d.values()
print(list(d.values())[-1])

4
